<a href="https://colab.research.google.com/github/perspectacular/Tensorflow_Certification_Preparation_Dicoding/blob/main/C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================================
# PROBLEM C5
#
# Build and train a neural network model using the Daily Min Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.19 on the normalized dataset.
# ============================================================================================

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [5]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [12]:
def solution_C5():
    data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
    urllib.request.urlretrieve(data_url, 'daily-min-temperatures.csv')

    time_step = []
    temps = []

    with open('daily-min-temperatures.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        step = 0
        for row in reader:
            temps.append(float(row[1]))
            time_step.append(int(step))
            step=step + 1
            
    series=np.array(temps)

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=2500

    time_train= time[:split_time]
    x_train= series[:split_time]
    time_valid= time[split_time:]
    x_valid= series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(
        x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    model=tf.keras.models.Sequential([
      tf.keras.layers.Conv1D( 
                          filters=60,
                          kernel_size=5,
                          strides=1,
                          padding='causal',
                          activation='relu',
                          input_shape= [None, 1]
                          ),
      tf.keras.layers.LSTM(60, return_sequences=True),
      tf.keras.layers.LSTM(60, return_sequences=True),
      tf.keras.layers.Dense(30, activation='relu'),
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(1)
    ])

    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 2e-5 * 10**(epoch / 20))
    optimizer = tf.keras.optimizers.SGD(momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
    model.fit(train_set, epochs=100, callbacks=[lr_schedule])
    return model

In [13]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_C5()
    model.save("model_C5.h5")

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/100
10/10 [==============================] - 8s 361ms/step - loss: 0.0748 - mae: 0.3584 - lr: 2.0000e-05
Epoch 2/100
10/10 [==============================] - 4s 356ms/step - loss: 0.0739 - mae: 0.3560 - lr: 2.2440e-05
Epoch 3/100
10/10 [==============================] - 4s 353ms/step - loss: 0.0726 - mae: 0.3523 - lr: 2.5179e-05
Epoch 4/100
10/10 [==============================] - 4s 362ms/step - loss: 0.0709 - mae: 0.3477 - lr: 2.8251e-05
Epoch 5/100
10/10 [==============================] - 4s 362ms/step - loss: 0.0691 - mae: 0.3424 - lr: 3.1698e-05
Epoch 6/100
10/10 [==============================] - 4s 354ms/step - loss: 0.0670 - mae: 0.3364 - lr: 3.5566e-05
Epoch 7/100
10/10 [==============================] - 4s 348ms/step - loss: 0.0648 - mae: 0.3298 - lr: 3.9905e-05
Epoch 8/100
10/10 [======================

In [15]:
model = tf.keras.models.load_model( 'model_C5.h5' )
input = model.input
print(input)
output = model.output
print(output)

KerasTensor(type_spec=TensorSpec(shape=(None, None, 1), dtype=tf.float32, name='conv1d_3_input'), name='conv1d_3_input', description="created by layer 'conv1d_3_input'")
KerasTensor(type_spec=TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None), name='dense_11/BiasAdd:0', description="created by layer 'dense_11'")
